In [1]:
main_dir = r'../../..'
data_path = main_dir+'/data'
import sys
sys.path.append(main_dir)
from typing import List, Set, Dict, Optional, Any, Tuple, Type, Union

from BayDS.lib.pipeline import *
from BayDS.lib.io import *
import os


In [2]:
data_dir = f'{main_dir}/Snapshots/1/06-NewIdJoint'
p = Pipeline(working_folder=f'{main_dir}/Snapshots/1/TimeAggs24.09')

p.add_node(LoaderNode, None, 'data',
           params={
               'input_directory': data_dir,
               'file': 'data.pkl'
           })
p.add_node(LoaderNode, None, 'numerical_columns',
           params={
               'input_directory': data_dir,
               'file': 'numerical_columns.yaml'
           })
p.add_node(LoaderNode, None, 'categorical_columns',
           params={
               'input_directory': data_dir,
               'file': 'categorical_columns.yaml'
           })

p.run()

---------------------------
0: LoaderNode [2019-09-24 21:59:43]
params:
 {'input_directory': '../../../Snapshots/1/06-NewIdJoint', 'file': 'data.pkl'}
---------------------------
1: LoaderNode [2019-09-24 21:59:50]
params:
 {'input_directory': '../../../Snapshots/1/06-NewIdJoint', 'file': 'numerical_columns.yaml'}
---------------------------
2: LoaderNode [2019-09-24 21:59:50]
params:
 {'input_directory': '../../../Snapshots/1/06-NewIdJoint', 'file': 'categorical_columns.yaml'}


In [3]:
p.add_node(AddTransactionFrequenciesNode, 
           ('data', 'numerical_columns', 'categorical_columns'),
           'transaction_frequencies'        
           )
p.run()
p.save_data(only_keys=['transaction_frequencies'])

---------------------------
3: AddTransactionFrequenciesNode [2019-09-24 22:00:24]
params:
 None
.
.
.
.
.
.
.
.
Saving transaction_frequencies


In [5]:
p.data['transaction_frequencies'].head()

,Transaction_freq_1d_past,Transaction_freq_2d_past,Transaction_freq_3d_past,Transaction_freq_7d_past,Transaction_freq_30d_past,Transaction_freq_5_past,Transaction_freq_5_center,Transaction_freq_10_past,Transaction_freq_10_center,Transaction_freq_100_past,Transaction_freq_100_center
TransactionID,,,,,,,,,,,
2987107,0.0,0.0,0.0,0.0,0.0,0.0,276.512130,0.0,470.435944,0.0,3231.052714
2987122,0.0,0.0,0.0,0.0,0.0,0.0,616.421111,0.0,837.770139,0.0,837.770139
2987161,0.0,0.0,0.0,0.0,0.0,0.0,2693.618611,0.0,4516.755389,0.0,6236.844722
2987174,0.0,0.0,0.0,0.0,0.0,0.0,179.819352,0.0,476.508833,0.0,2665.241544
2987225,0.0,0.0,0.0,0.0,0.0,0.0,15.158796,0.0,21.757944,0.0,227.386433


In [10]:
test = p.data['data']
test = test[test['new_card_id'] == 0][['Date']]

In [13]:
test.join(p.data['transaction_frequencies']).head()

,Date,Transaction_freq_1d_past,Transaction_freq_2d_past,Transaction_freq_3d_past,Transaction_freq_7d_past,Transaction_freq_30d_past,Transaction_freq_5_past,Transaction_freq_5_center,Transaction_freq_10_past,Transaction_freq_10_center,Transaction_freq_100_past,Transaction_freq_100_center
TransactionID,,,,,,,,,,,,
2987107,2017-12-02 00:30:58,0.0,0.0,0.000000,0.000000,0.000000,0.000000,276.512130,0.000000,470.435944,0.000000,3231.052714
3052707,2017-12-18 01:02:45,0.0,0.0,0.000000,0.000000,192.264861,192.264861,332.289514,192.264861,575.037454,192.264861,3231.052714
3063714,2017-12-20 13:31:22,0.0,0.0,30.238472,30.238472,276.512130,276.512130,470.435944,276.512130,662.661746,276.512130,3231.052714
3077305,2017-12-22 20:08:16,0.0,0.0,27.307500,58.522963,332.289514,332.289514,305.515222,332.289514,767.783854,332.289514,3231.052714
3160462,2018-01-13 15:32:16,0.0,0.0,0.000000,0.000000,203.515208,470.435944,405.813833,470.435944,851.911667,470.435944,3231.052714


## Node contents were made here

In [ ]:
all_cols = set(p.data['data'].columns)
missing = all_cols - ( set(p.data['categorical_columns']) | set(p.data['numerical_columns'] ))
try:
    assert len(missing) == 0
except AssertionError:
    print(yaml.dump(list(missing)))    
spare =( set(p.data['categorical_columns']) | set(p.data['numerical_columns'] )) -  all_cols
try:
    assert len(spare) == 0
except AssertionError:
    print(yaml.dump(list(spare)))

In [ ]:
data = p.data['data'][['Date', 'new_card_id']]

In [ ]:
group_by_feature = 'new_card_id'
check_data = data.reset_index().set_index([group_by_feature, 'Date'])
duplicate_transactions = check_data[check_data.index.duplicated()]['TransactionID'].values
while len(duplicate_transactions) > 0:
    print(f"Found {len(duplicate_transactions)} duplicate transactions")
    for itid, tid in enumerate(duplicate_transactions):
        print(itid)
        q = data.loc[tid]
        date = q['Date']
        card_id = q[group_by_feature]
        alldup = data[data['Date'] == date]
        alldup = alldup[alldup[group_by_feature] == card_id]
        #     print(alldup.index)
        for it, idx in enumerate(alldup.index):
            #         print(idx)
            data.loc[idx, 'Date'] += pd.Timedelta(seconds=it)
    check_data = data.reset_index().set_index([group_by_feature, 'Date'])
    duplicate_transactions = check_data[check_data.index.duplicated()]['TransactionID'].values

In [ ]:
data.loc[:,'hours'] =  (data.Date - data.Date.iloc[0]).dt.total_seconds() / 3600

In [ ]:
# transaction_velocity_agg = lambda x: (x - x.iloc[0] ).mean()
def transaction_velocity_agg(x):
    return ((x - x.iloc[0])).mean()

In [ ]:
small = data[data.new_card_id < 3]

In [ ]:
small.head()

In [ ]:
# gb = df[df['new_card_id'] == 0] 
gb = small.reset_index().groupby(['new_card_id'])
q = gb.rolling(3, on='Date', min_periods=1, center=True)['hours'].agg(transaction_velocity_agg)


In [ ]:
q.head()

In [ ]:
small = small.reset_index().set_index(['new_card_id', 'Date'])
small['Transaction_freq_3_center'] = q

In [ ]:
small = small.set_index('TransactionID')

In [ ]:
small

In [ ]:
df.reset_index()

In [ ]:
q.rename(columns={'Date':'D1'})

In [ ]:
pd.DataFrame(q)

In [ ]:
q.reset_index()

In [ ]:
gb.head()

In [ ]:
# pd.to_timedelta(gb['Date'])
q = (gb['Date'] - gb['Date'].iloc[0] ).dt.total_seconds().mean() / 3600
q

In [ ]:
gb['Date'].agg(lambda x: (x - x.iloc[0] ).dt.total_seconds().mean() / 3600)